In [1]:
import heapq
from collections import defaultdict, deque
from typing import List

**121. Best time to buy and sell stock**


In [3]:
"""
2nd attempt after a long time because i am doing the 2d dp version of this (Dec 18, 2024)
"""


class Solution:
    def maxProfit(self, prices: List[int]) -> int:  # 53% time, 5% memory
        """
        Can have a left pointer at the current minimum value seen
        Right pointer just moves until the end and computes max profit
        """

        left = 0
        res = 0
        for right in range(len(prices)):
            if prices[right] < prices[left]:
                left = right
            else:
                res = max(res, prices[right] - prices[left])
        return res


maxProfit = Solution()
(
    maxProfit.maxProfit([7, 1, 5, 3, 6, 4]),
    maxProfit.maxProfit([7, 6, 4, 3, 1]),
)

(5, 0)

In [3]:
class Solution:  # 80% time, 25% memory
    def maxProfit(self, prices: List[int]) -> int:
        # slide window while updating max difference
        # update left when you reach a new smallest value
        # increment right
        # variable size window
        # has to be (0, 0) because an array of length 1 is a possible input
        l, r = 0, 0
        res = max(0, prices[r] - prices[l])
        while r < len(prices):
            if prices[r] < prices[l]:
                l = r
            res = max(res, prices[r] - prices[l])
            r += 1
        return res


# NeetCode solution, doesn't use pointers, but same logic. For loops are faster than while loops in python though
# class Solution:
#     def maxProfit(self, prices: List[int]) -> int:
#         res = 0

#         lowest = prices[0]
#         for price in prices:
#             if price < lowest:
#                 lowest = price
#             res = max(res, price - lowest)
#         return res

maxProfit = Solution()
print(maxProfit.maxProfit([7, 1, 5, 3, 6, 4]))
print(maxProfit.maxProfit([7, 6, 4, 3, 1]))

5
0


**3. Longest substring witout repeating characters**


In [12]:
class Solution:  # 90% time, 25% memory
    def lengthOfLongestSubstring(self, s: str) -> int:
        # sliding window using a queue (just gonna use a list)
        # i.e. variable size window
        hash_set = set()  # keep track of characters in queue (O(1) search)
        queue = []
        res = 0
        for char in s:
            if char in hash_set:
                res = max(res, len(queue))
                while queue:
                    hash_set.remove(q := queue.pop(0))
                    if q == char:
                        break
            queue.append(char)
            hash_set.add(char)

        return max(res, len(queue))


# NeetCode Solution: # two pointers instead of a double queue
# class Solution:
#     def lengthOfLongestSubstring(self, s: str) -> int:
#         charSet = set()
#         l = 0
#         res = 0

#         for r in range(len(s)):
#             while s[r] in charSet:
#                 charSet.remove(s[l])
#                 l += 1
#             charSet.add(s[r])
#             res = max(res, r - l + 1)
#         return res


lengthOfLongestSubstring = Solution()
lengthOfLongestSubstring.lengthOfLongestSubstring("abcabcbbegh")

4

**424. Longest Repeating Character replacement**


In [51]:
class Solution:
    def characterReplacement(self, s: str, k: int) -> int:  # 38% time, 25% memory
        """
        You are given a string s and an integer k. You can choose any character of the string and change it to any other uppercase English character.
        You can perform this operation at most k times.
        Return the length of the longest substring containing the same letter you can get after performing the above operations.
        """
        res = 0
        char_count = defaultdict(
            int
        )  # hashmap with number of occurrences for a character in current stack
        # when k is less than the amount of occurrences of letter that are not the most common, move left pointer until that changes
        left = 0
        for i in range(len(s)):
            char_count[s[i]] += 1
            if (i + 1 - left) - max(
                char_count.values()
            ) <= k:  # length of stack - most common <= number of replacements
                res = max(res, i + 1 - left)
                continue
            while (i + 1 - left) - max(char_count.values()) > k:
                char_count[s[left]] -= 1
                left += 1
        return res

    # better solution from NeetCode keeping store of maxFrequency in stack
    # since the max result occurs at the maxFrequency, you don't need maxFrequency to be accurate at all times
    #   it just needs to true about the historical or current maxFrequency
    # length - maxF <= k, k is constant so get highest result by increasing length and maxF
    def characterReplacementNeetCode(
        self, s: str, k: int
    ) -> int:  # 88% time, 12% memory
        res = 0
        l = 0
        count = {}
        maxF = 0
        # only checking cases where res can actually be updated (when length - maxF <= k)
        # so the only intermediate logic for moving left pointer is when that condition is not met
        for r in range(len(s)):
            count[s[r]] = count.get(s[r], 0) + 1
            maxF = max(maxF, count[s[r]])
            while (
                r - l + 1
            ) - maxF > k:  # maxF doesnt get decremented because of the math logic
                count[s[l]] -= 1
                l += 1
            res = max(res, r - l + 1)
        return res

    def characterReplacementMarch18(self, s: str, k: int) -> int:
        maxFreq = 0
        # two pointers, find substring with the largest frequency of same characters
        # this substring + k is the answer
        left = 0
        count = {}
        for r in range(len(s)):
            count[s[r]] = count.get(s[r], 0) + 1
            maxFreq = max(maxFreq, count[s[r]])
            while maxFreq + k < (r - left + 1):
                count[s[left]] -= 1
                left += 1
        return min(maxFreq + k, len(s))  # edge case


characterReplacement = Solution()
characterReplacement.characterReplacementNeetCode("ABAA", 0)

2

**567. Permutation in strings**


In [69]:
class Solution:  # 74% time, 8% memory
    def checkInclusion(self, s1: str, s2: str) -> bool:
        if len(s1) > len(s2):
            return False
        count_s1 = [0] * 26  # 26 characters only
        count_s2 = [0] * 26  # 26 characters only
        matches = 0  # the return condition
        for c in range(len(s1)):  # initializing the first window
            count_s1[ord(s1[c]) - ord("a")] += 1
            count_s2[ord(s2[c]) - ord("a")] += 1
            # just populating based on 'a' -> 0, 'z' -> 25 index

        # first and only pass through entire hashmap for both:
        for i in range(26):
            if count_s1[i] == count_s2[i]:
                matches += 1

        l = 0
        for r in range(len(s1), len(s2)):
            if matches == 26:
                return True
            count_s2[ord(s2[l]) - ord("a")] -= 1
            if count_s2[ord(s2[l]) - ord("a")] + 1 == count_s1[ord(s2[l]) - ord("a")]:
                matches -= 1
            elif count_s2[ord(s2[l]) - ord("a")] == count_s1[ord(s2[l]) - ord("a")]:
                matches += 1

            l += 1
            count_s2[ord(s2[r]) - ord("a")] += 1
            if count_s2[ord(s2[r]) - ord("a")] - 1 == count_s1[ord(s2[r]) - ord("a")]:
                matches -= 1
            elif count_s2[ord(s2[r]) - ord("a")] == count_s1[ord(s2[r]) - ord("a")]:
                matches += 1

        # this was the issue, there was no final check for the return condition:
        return matches == 26


checkInclusion = Solution()
checkInclusion.checkInclusion("ab", "eidbaoo")

True

**76. Minimum Window Substring**


In [42]:
class Solution(object):
    def minWindow(self, s: str, t: str) -> str:  # 86% time, 45% memory
        # using NeetCode's have and need method without seeing the code
        if t == "":
            return ""
        hashNeed, hashHave = {}, {}
        for c in t:
            hashNeed[c] = 1 + hashNeed.get(c, 0)
            hashHave[c] = 0

        # This was the issue, I initialized `need` as `len(t)` but it has to be the number of unique characters in `t`
        have, need = 0, len(hashNeed)

        # len(window), start index, end index
        result = [float("inf"), None, None]
        l = 0
        for r in range(len(s)):
            if s[r] in hashNeed:
                hashHave[s[r]] += 1
                if hashHave[s[r]] == hashNeed[s[r]]:
                    have += 1
            while have == need:
                if r - l < result[0]:
                    result = [
                        r - l,
                        l,
                        r,
                    ]  # omit plus 1 in `r-l` for all since they cancel
                if s[l] in hashNeed:
                    hashHave[s[l]] -= 1
                    if hashHave[s[l]] < hashNeed[s[l]]:
                        have -= 1
                l += 1

        if result[1] is None:
            return ""
        return s[result[1] : result[2] + 1]


minWindow = Solution()
minWindow.minWindow("ADOBECODEBANC", "ABC")
# minWindow.minWindow("a", "b")

'BANC'

**Maximum Width Ramp**


In [2]:
class Solution:
    def maximumWidthRamp(self, heights: list[int]):
        suffix_max_height = [0] * len(heights)
        suffix_max_height[-1] = heights[-1]
        for i in reversed(range(len(heights) - 1)):
            suffix_max_height[i] = max(suffix_max_height[i + 1], heights[i])

        left, right = 0, 0
        res = 0
        while right < len(heights):
            if heights[left] > suffix_max_height[right]:
                left += 1
            elif heights[right] < suffix_max_height[right]:
                right += 1
            elif (heights[right] == suffix_max_height[right]) and (
                heights[right] >= heights[left]
            ):
                res = max(res, right - left)
                right += 1
            else:
                left += 1
        return res


S = Solution()
print(
    S.maximumWidthRamp([6, 0, 8, 1, 2, 5]),
    S.maximumWidthRamp([6, 0, 8, 1, 2, 6, 9, 10]),
)

4 7


**239. Maximum Sliding Window**

Type of Decreasing Monotonic Queue problem


In [52]:
class Solution:  # 10% time, 6% memory
    def maxSlidingWindow(self, nums: List[int], k: int) -> List[int]:
        """
        for every k-sized window moving from left to right return the max

        I'm thinking just use a maxHeap and when the index is out of bounds pop
        """

        # the index will be kept positive because I want the leftmost index to be prioritized
        # and pooped before the rightmost one if the numbers are duplicated
        maxHeap = [(-nums[i], i) for i in range(k)]
        heapq.heapify(maxHeap)
        res: List[int] = []
        for i in range(k, len(nums) + 1):
            res.append(-maxHeap[0][0])
            while maxHeap and maxHeap[0][1] <= i - k:
                heapq.heappop(maxHeap)
            if i == len(nums):
                break
            heapq.heappush(maxHeap, (-nums[i], i))

        return res

    def maxSlidingWindowDeque(
        self, nums: List[int], k: int
    ) -> List[int]:  # 39% time, 17% memory
        """
        NeetCode solution involves a deque
        Where at any given moment in time the deque is a decreasing monotonic queue
        such that the leftmost integer is the max of the current window being evaluated
        (need to ensure to remove values once they are out of bounds)
        """

        res = []
        dq = deque()
        # setup the first k window deque
        for i in range(len(nums)):
            while dq and nums[i] > dq[-1][0]:
                dq.pop()
            dq.append((nums[i], i))
            if i >= k - 1:
                if dq[0][1] <= i - k:
                    dq.popleft()
                res.append(dq[0][0])

        return res

    def maxSlidingWindowNeetCode(self, nums: List[int], k: int) -> List[int]:
        output = []
        q = deque()  # index
        l = r = 0

        while r < len(nums):
            while q and nums[q[-1]] < nums[r]:
                q.pop()
            q.append(r)

            if l > q[0]:
                q.popleft()

            if (r + 1) >= k:
                output.append(nums[q[0]])
                l += 1
            r += 1

        return output


maxSlidingWindow = Solution()
print(maxSlidingWindow.maxSlidingWindowDeque([1, 3, -1, -3, 5, 3, 6, 7], 3))
print(maxSlidingWindow.maxSlidingWindowDeque([1], 1))
print(maxSlidingWindow.maxSlidingWindowDeque([9, 10, 9, -7, -4, -8, 2, -6], 5))

[3, 3, 5, 5, 6, 7]
[1]
[10, 10, 9, 2]


**3208. Alternating Groups II**


In [ ]:
class Solution:
    # 33% time, 58% memory
    def numberOfAlternatingGroups(self, colors: List[int], k: int) -> int:
        """
        Initialize first window of size k and then just go until you evaluate all windows starting at every index in colors
        Will need to mod the indexing in order to avoid going index out of bounds

        Actually, no need to initialize. Just try to build the window, resetting every time it stops alternating, and increment reuslt whenver you seucced.
        """
        result = 0
        left = 0
        right = 1
        n = len(colors)
        while left < n:
            if colors[right % n] == colors[(right - 1) % n]:
                # mod n because colors is a circle
                left = right
            elif right - left == k - 1:
                result += 1
                left += 1
            right += 1
        return result


numberOfAlternatingGroups = Solution()
print(numberOfAlternatingGroups.numberOfAlternatingGroups([0, 1, 0, 1, 0, 1], 3))
print(numberOfAlternatingGroups.numberOfAlternatingGroups([0, 1, 0, 0, 1, 0, 1], 6))
print(numberOfAlternatingGroups.numberOfAlternatingGroups([1, 1, 0, 1], 4))

0
1
1
1
0


**3306. Count of Substrings Containing Every Vowel and K Consonants II**


In [ ]:
class Solution:
    # 93% time, 7% memory
    def countOfSubstrings(self, word: str, k: int) -> int:
        """
        Just gonna recreate the editorial's first solution without looking at it.
        Basically same approach as my first attempt but they have a proper way of handling trailing vowels on the right
        by using a nextConsonant array which has the index of the next consonant following the current index
        """
        n = len(word)
        nextConsonant = [0] * n
        nextConsonant[-1] = n
        for i in reversed(range(1, n)):
            if word[i] in "aeiou":
                nextConsonant[i - 1] = nextConsonant[i]
            else:
                nextConsonant[i - 1] = i

        vowels = {}
        consonants = 0
        left = right = 0
        result = 0
        while right < n:
            c = word[right]
            if c in "aeiou":
                vowels[c] = vowels.get(c, 0) + 1
            else:
                consonants += 1

            while consonants > k:
                # need to contract from the left because we cannot meet the requirements otherwise
                c = word[left]
                if c in "aeiou":
                    if vowels[c] == 1:
                        del vowels[c]
                    else:
                        vowels[c] -= 1
                else:
                    consonants -= 1
                left += 1

            while consonants == k and len(vowels) == 5:
                # this is how many more strings we can make with trailing vowels at this (left, right)
                result += nextConsonant[right] - right
                c = word[left]
                if c in "aeiou":
                    if vowels[c] == 1:
                        del vowels[c]
                    else:
                        vowels[c] -= 1
                else:
                    consonants -= 1
                left += 1

            right += 1

        return result


countOfSubstrings = Solution()
print(countOfSubstrings.countOfSubstrings("aeioqq", 1))
print(countOfSubstrings.countOfSubstrings("aeiou", 0))
print(countOfSubstrings.countOfSubstrings("ieaouqqieaouqq", 1))

0
1
3


**1358. Number of Substrings Containing All Three Characters**


In [ ]:
class Solution:
    """
    The proper solution is not something I understand just by looking at code, will have to hope that neetcode covers it
    """

    # 24% time, 39% memory
    def numberOfSubstrings(self, s: str) -> int:
        """
        It says at least one occurrence of all the letters a, b, c
        So the moment this condition is filled len(s) - right is how many results you just found
        Then just shift left and rinse repeat
        """
        left = 0
        freq = [0, 0, 0]
        a, count = ord("a"), 0
        n = len(s)
        result = 0
        for right, c in enumerate(s):
            if c in "abc":
                freq[ord(c) - a] += 1
                count += int(freq[ord(c) - a] == 1)

            while count == 3:
                result += n - right
                c = s[left]
                if c in "abc":
                    freq[ord(c) - a] -= 1
                    count -= int(freq[ord(c) - a] == 0)
                left += 1
        return result


numberOfSubstrings = Solution()
print(numberOfSubstrings.numberOfSubstrings("abcabc"))
print(numberOfSubstrings.numberOfSubstrings("aaacb"))
print(numberOfSubstrings.numberOfSubstrings("abc"))

10
3
1
